### DSCI 100 Group Project: Predicting Disease Diagnoses

### Introduction

The Multiple Disease Prediction dataset uses various health-related parameters (such as glucose and cholesterol levels, insulin levels, and BMI) to check the health of a person - whether they are healthy or afflicted with a specific disease (Diabetes, Thalasse, Anemia, or Thromboc). In this project, we set out to create a classification model to examine to what accuracy we can predict the disease a person with certain parameters can be diagnosed with.

### Exploratory Data Analysis

In [2]:
### Run this cell before continuing. 
library(tidyverse)
library(repr)
library(tidymodels)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.5     ✔ workflows    1.1.3
✔ modeldata    1.2.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.1     ✔ yardstick    1.2.0
✔ recipes      1.0.8     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

In [4]:
multiple_diseases <- read_csv("data/blood_samples_dataset.csv")
print(multiple_diseases, n = 10)

Rows: 2351 Columns: 25
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (1): Disease
dbl (24): Glucose, Cholesterol, Hemoglobin, Platelets, White Blood Cells, Re...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


# A tibble: 2,351 × 25
   Glucose Cholesterol Hemoglobin Platelets `White Blood Cells`
     <dbl>       <dbl>      <dbl>     <dbl>               <dbl>
 1   0.740      0.650       0.714    0.868               0.687 
 2   0.122      0.0231      0.945    0.905               0.508 
 3   0.453      0.116       0.545    0.401               0.295 
 4   0.137      0.0156      0.420    0.191               0.0812
 5   0.177      0.752       0.972    0.785               0.444 
 6   0.898      0.259       0.933    0.353               0.374 
 7   0.354      0.758       0.755    0.0126              0.228 
 8   0.399      0.508       0.777    0.0963              0.926 
 9   0.722      0.300       0.488    0.201               0.402 
10   0.399      0.508       0.777    0.0963              0.926 
# ℹ 2,341 more rows
# ℹ 20 more variables: `Red Blood Cells` <dbl>, Hematocrit <dbl>,
#   `Mean Corpuscular Volume` <dbl>, `Mean Corpuscular Hemoglobin` <dbl>,
#   `Mean Corpuscular Hemoglobin Concentration` <

### Methods

Some methods

### Expected Outcomes

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.5     ✔ workflows    1.1.3
✔ modeldata    1.2.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.1     ✔ yardstick    1.2.0
✔ recipes      1.0.8     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

In [10]:
blood_samples <- read_csv("data/blood_samples_dataset.csv")

Rows: 2351 Columns: 25
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (1): Disease
dbl (24): Glucose, Cholesterol, Hemoglobin, Platelets, White Blood Cells, Re...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [15]:
unique_diseases <- select(blood_samples, Disease) |>
            distinct()
# plot <- ggplot(blood_samples, aes(

Disease
<chr>
Healthy
Diabetes
Thalasse
Anemia
Thromboc
